# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import pickle

In [110]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [194]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [ ]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[6089:6999]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

In [107]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,isbn,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,9791158510787,[8.8],"[45%, 55%, 0%, 0%, 0%]","[일개미 자서전, 여왕개미로 태어난 줄 알았지만., 일개미들의 이야기 : 일개미 자...","[북*더, w********k, 까**걀, S*********a, m********...","[5점, 3점, 5점, 5점, 5점, 5점, 4점, 4점, 4점, 4점]",[\n『일개미 자서전』묘하게 빠져드는 책이다. 글을 참 맛깔나게 잘 쓰신다는 생각이...
1,9788998791636,[7.0],"[0%, 50%, 50%, 0%, 0%]","[이것은 공감 아닌 공감 - 독서광의 모험은 끝나지 않아!, 독서광들의 대담_독서광...","[k*******n, 태**빠]","[4점, 3점]",[\n대학 시절... 책을 사러 정문 앞 서점에 찾아간 적이 있었다.그곳에서 사장님...
2,9788968571800,[9.8],"[100%, 0%, 0%, 0%, 0%]","[『마음에도 길이 있어요』내가 꾸는 꿈, 마음에도 길이 있어요, 마음에도 길이 있어...","[신*****리, j********1, 굿*, 뽀*맘, l**********6, c...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n \n \n \n1.\n \n하나가 빠지면 서운하지 않을까? 하나가 더해...
3,9791156225430,[],[],[],[],[],[]
4,9791164353071,[9.5],"[100%, 0%, 0%, 0%, 0%]","[통합교육 그 안에 숨겨진 보물찾기, 함께 살아가기]","[h*e, 조*강]","[5점, 5점]",[\n이 책은 통합교육의 실제 사례들이 담겨져 있는 교직 일기이다. 제목이 참 흥미...
5,9791195749157,[10.0],"[100%, 0%, 0%, 0%, 0%]","[마음은 홍차, 마음은 홍차, 홍차 한 잔에 담긴 위로 한 모금하세요 ㅎ, 홍차의 ...","[빛***석, a****l, s*******g, s*******i]","[5점, 5점, 5점, 5점]",[\n 너무 앙증맞고 사랑스러운 그림에세이 책이다. 오른쪽 엽서와 비슷한 크기의 책...
6,9788949716022,[],[],[],[],[],[]
7,9788983717979,[6.0],"[0%, 0%, 100%, 0%, 0%]",[리뷰],[l******5],[3점],[\n인디언 명언이 인터넷에 즐비하다. 이는 꽤 오래전부터 많은 사람들에게 깊은 영...
8,9788954616355,[9.0],"[83%, 0%, 17%, 0%, 0%]","[자연스러운 그 사람, 오원근..그 사람의 삶에 공감하다!, 솔직함에 빠져 밤새 책...","[t******q, j****k, B*******e, t*****i, 구**모, 모*루]","[5점, 5점, 5점, 5점, 5점, 3점]","[\n노란 색 바탕에 순수한 소년의 눈망울을 지닌 오원근 변호사의 사진, 그리고 '..."
9,9791159350818,[9.2],"[76%, 18%, 6%, 0%, 0%]","[산은 늘 거기 그렇게 있어 주었다, 산이 좋아졌어, 직장인 등산가 에세이, 산을 ...","[나*이, 산*람, s****u, 아**드, h****n, s***h, k*****...","[4점, 5점, 5점, 4점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n\n \n산은 나를 늘 행복하게 하는 공간이다. 그 곳에 있으면 세상의 일들이...


In [109]:
user.to_pickle('user_6089-6088.pkl', protocol = 4)

In [108]:
for i in range(6089,6999):
    if isbn13[i] == '9788966000012':
        print(i)

6088


In [ ]:
user1 = pd.read_pickle('user_0-1115.pkl')
user2 = pd.read_pickle('user_1116-1161.pkl')
user3 = pd.read_pickle('user_1162-1170.pkl')
user4 = pd.read_pickle('user_1171-2596.pkl')

In [ ]:
df = pd.concat([user1, user2,user3, user4],ignore_index=True)
df.to_pickle('user_0-2596.pkl', protocol = 4)

In [39]:
pd.concat([user1, user2])

,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,[9.6],"[75%, 25%, 0%, 0%, 0%]","[지금 잘 살아야 하는 이유에 대한 통찰 [나는 환생을 믿지 않았다], 명료함을 불...","[하*애, s****a, j**a, H************2]","[5점, 4점, 5점, 5점]","[\n이상민, 조우종이 진행하는 <차트를 달리는 남자>, 전 세계 별난 이야기들만 ..."
1,[10.0],"[100%, 0%, 0%, 0%, 0%]",[[약삭빠르게 온천] 고독한 미식가의 도심 온천 여행기],[키*],[5점],[\n\n \n드라마로 유명한 일본 만화 <고독한 미식가>의 원작자 구스미 마사유키...
2,[9.8],"[94%, 6%, 0%, 0%, 0%]","[고독함과 마주 할 때에 어떻게 할까?/피카, 세상에 나 혼자라고 느껴질 때, 세상...","[나*이, 보**쥬, 러***니, 샤*라, 폭**상, 카*라, 줄**북, 설*산, ...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n이 책은?\n \n스스로 고독과 마주친 사람의 고독에 대한 대면을 그리고 있...
3,[9.8],"[92%, 8%, 0%, 0%, 0%]","[세상 끝에서 춤추다, 『세상 끝에서 춤추다』 르 귄을 만든 시간의 기록, 르귄의 ...","[모*, 누*, j*********n, 키*, 깔**s, y******k, o***...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n\n \n도 서: 세상 끝에서 춤추다\n \n저 자: 어슐러 K.르 귄\n \...
4,[8.6],"[50%, 44%, 0%, 6%, 0%]","[나의 미래를 예측하는 유일한 방법은 나의 오늘을 보는 것이다, ""데뷔의 순간"" 후...","[s*******r, e*****t, c*****1, r******2, 알***투,...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 4점, 4점]","[\n한때 영화계를 기웃거렸던 적이 있다. 대학 시절, 연출자가 되어 보겠다고 시나..."
...,...,...,...,...,...,...
42,[9.5],"[86%, 12%, 2%, 0%, 0%]","[[나도 모르게 생각한 생각들] 2021_095, 나도 모르게_017 (나도 모르게...","[사*님, J*y, 책****벤, n********d, 짓*, i*****6, 긴*...","[4점, 4점, 4점, 5점, 5점, 5점, 5점, 3점, 5점, 5점]",[\n2021_095\n \n\n\n읽은날 : 2021.12.23~2021.12.2...
43,[8.6],"[46%, 46%, 9%, 0%, 0%]","[이렇게 귀여운 할매를 보았나!, 사는게 뭐라고 - 울엄마는 지겹다고 하신다 요즘 ...","[박*리, m**a, 콩*, 부***주, j***7, 오***객, 김*진, 껌***...","[5점, 5점, 5점, 4점, 5점, 4점, 4점, 4점, 5점, 5점]",[\n책 제목을 보고 맥주 광고를 떠 올렸다. 딱히 그 배우를 좋아하는 것은 아닌데...
44,[9.0],"[79%, 7%, 14%, 0%, 0%]","[서평보다 먼저 빈대떡을 부쳤다, 역시 띵시리즈, [맛] 엄마 박완서의 부엌, 엄마...","[t*****1, S***e, 싱*, 책****벤, 숲*래, n**t, 보*비, 잔...","[5점, 5점, 5점, 5점, 3점, 5점, 5점, 5점, 5점, 5점]",[\n고등학교에는 0교시가 있었다. 교실 맨 뒷자리에 앉았던 나는 대부분이 비몽사몽...
45,[10.0],[],[],[],[],[]
